# Trading @ Georgia Tech Exchange Client

### Constants

In [1]:
# TODO: fill these constants
WORKING_DIRECTORY = '/content/drive/MyDrive/TradingClient'
USERNAME = ""
API_KEY = ""

# do not modify
RATE_LIMIT = 15
URI = 'ec2-3-16-107-184.us-east-2.compute.amazonaws.com'
URL = f"http://{URI}:8080"
WS_URL = f"ws://{URI}:8080/exchange-socket"

In [2]:
import os
from google.colab import drive
os.environ['WORKING_DIRECTORY'] = WORKING_DIRECTORY
drive.mount('/content/drive')

Mounted at /content/drive


In [3]:
!echo $WORKING_DIRECTORY
!cd $WORKING_DIRECTORY

/content/drive/MyDrive/TradingClient


### Importing Dependencies

In [4]:

!pip install sortedcontainers aiohttp websockets pandas numpy


In [5]:

from __future__ import annotations

import aiohttp
import asyncio
import contextlib
import copy
import json
import time
import traceback
import urllib.request
from abc import ABC
from abc import abstractmethod
from collections import deque
from dataclasses import dataclass
from enum import Enum
from typing import Any

import nest_asyncio
import websockets
from sortedcontainers import SortedDict

nest_asyncio.apply()

import sys
sys.path.append(WORKING_DIRECTORY)
from client_utils import *


### Strategy Implementation

In [6]:
class TestStrategy(Strategy):
    def __init__(self, quoter: Prioritizer, shared_state: SharedState):
        super().__init__(quoter, shared_state)
        self._cnt = 1

    async def on_orderbook_update(self) -> None:
        print("Orderbook update", self._cnt, time.time())
        # asyncio.create_task(self._quoter.remove_all())
        asyncio.create_task(self._quoter.place_limit(ticker="A", volume=1, price=50+self._cnt, is_bid=True))
        asyncio.create_task(self._quoter.place_limit(ticker="A", volume=1, price=950-self._cnt, is_bid=False))
        # asyncio.create_task(self._quoter.place_market(ticker="A", volume=1, is_bid=True))
        # asyncio.create_task(self._quoter.place_market(ticker="A", volume=1, is_bid=False))
        self._cnt += 1

    async def on_portfolio_update(self) -> None:
        print("Portfolio update", self._cnt, time.time())
        print(self._shared_state.portfolio.positions)
        pass

In [7]:
async def start_strategy() -> None:
    """
    Async method to start a strategy.
    Returns: None

    """
    client = TradingClient(
        http_endpoint=URL,
        ws_endpoint=WS_URL,
        username=USERNAME,
        api_key=API_KEY,
    )
    shared_state = client.shared_state
    prioritizer = Prioritizer(rate_limit=RATE_LIMIT, trading_client=client)

    strategy: Strategy = TestStrategy(quoter=prioritizer, shared_state=shared_state)

    client.set_strategy(strategy=strategy)

    await strategy.start()

    await asyncio.sleep(1000000)


async def main() -> None:
    """
    Main async method for running all client tasks as asynchronous coroutines.
    Returns: None

    """
    tasks: list[asyncio.Task[None]] = [asyncio.create_task(start_strategy())]
    try:
        results = await asyncio.gather(
            *tasks,
            return_exceptions=True,
        )
        print(results)
    except Exception as e:
        print("Exception in main", e)
        traceback.print_exc()
        for task in tasks:
            task.cancel()
        await asyncio.gather(*tasks, return_exceptions=True)


if __name__ == "__main__":
    asyncio.run(main())

[URLError(ConnectionRefusedError(111, 'Connection refused'))]
